In [3]:
from rtree import RTree, RTreeNode, Rectangle


In [4]:

from typing import List, Optional, Generator, Tuple, Union

class RTreeChild(RTree):
    def __init__(self, max_children: int, min_fill: float):
        super().__init__(max_children, min_fill)
    
    
        
    #@override
    def _handle_overflow(self, node: RTreeNode) -> None:
        """
        Si node excede capacidad, realiza reinserción diferida y split.
        """
        count = len(node.data_points) if node.is_leaf else len(node.children)
        if count <= self.M:
            return
        # Reinserción diferida: eliminar y guardar
        
        entries = self._pick_reinsert_entries(node)
        
        for e in entries:    
            if isinstance(e, Rectangle):
                e.show()
        
        for e in entries:
            if node.is_leaf:
                node.data_points.remove(e)  # type: ignore
            else:
                node.children.remove(e)  # type: ignore
        
        node._recalculate_mbr()
        
        # Primero split del nodo reducido
        self._split_node(node)
        
        # Luego reinserción de todas las entradas
        for e in entries:
            if isinstance(e, Rectangle):
                self.insert(e)
            else:
                print("Llega aca")
                for r in (e.data_points if e.is_leaf else e.children):
                    self.insert(r)
                    
    def _pick_reinsert_entries(self, node: RTreeNode) -> List[Union[Rectangle, RTreeNode]]:
        """
        Selecciona las k entradas más alejadas del centro de MBR para reinserción.
        """
        center_x = (node.mbr.min_x + node.mbr.max_x) / 2  # type: ignore
        center_y = (node.mbr.min_y + node.mbr.max_y) / 2  # type: ignore
        
        
        def dist(entry):
            r = entry if isinstance(entry, Rectangle) else entry.mbr  # type: ignore
            cx = (r.min_x + r.max_x) / 2
            cy = (r.min_y + r.max_y)     / 2
            return (cx - center_x)**2 + (cy - center_y)**2
        entries = node.data_points if node.is_leaf else node.children
        sorted_entries = sorted(entries, key=dist, reverse=True)
        return sorted_entries[:self.reinsert_count]
    
    
    def _split_node(self, node: RTreeNode) -> None:
        """
        Divide node usando heurística cuadrática y garantiza min_fill.
        Eficiencia: recálculo único al final.
        """
        items = node.data_points if node.is_leaf else node.children  # type: ignore
        seed1, seed2 = self._pick_seeds_quadratic(items)
        g1 = RTreeNode(is_leaf=node.is_leaf)
        g2 = RTreeNode(is_leaf=node.is_leaf)
        g1.add_data_point(seed1, recalc=False) if node.is_leaf else g1.add_child(seed1, recalc=False)
        g2.add_data_point(seed2, recalc=False) if node.is_leaf else g2.add_child(seed2, recalc=False)
        remaining = [it for it in items if it not in (seed1, seed2)]
        
        while remaining:
            size1 = len(g1.data_points) if g1.is_leaf else len(g1.children)
            size2 = len(g2.data_points) if g2.is_leaf else len(g2.children)
            
            if size1 + len(remaining) == self.m:
                for it in remaining:
                    g1.add_data_point(it, recalc=False) if node.is_leaf else g1.add_child(it, recalc=False)
                break
            
            if size2 + len(remaining) == self.m:
                for it in remaining:
                    g2.add_data_point(it, recalc=False) if node.is_leaf else g2.add_child(it, recalc=False)
                break
            
            it = remaining.pop(0)
            rect = it if node.is_leaf else it.mbr  # type: ignore
            
            r1 = rect if g1.mbr is None else Rectangle.union(g1.mbr, rect)
            
            rect = it if node.is_leaf else it.mbr  # type: ignore
            r2 = rect if g2.mbr is None else Rectangle.union(g2.mbr, rect)
            
            d1 = r1.area() - (g1.mbr.area() if g1.mbr else 0)
            d2 = r2.area() - (g2.mbr.area() if g2.mbr else 0)
            
            target = g1 if d1 < d2 else g2
            if node.is_leaf:
                target.add_data_point(it, recalc=False)  # type: ignore
            else:
                target.add_child(it, recalc=False)  # type: ignore
                
        g1._recalculate_mbr()
        g2._recalculate_mbr()
        parent = node.parent
        
        if parent is None:
            new_root = RTreeNode(is_leaf=False)
            new_root.add_child(g1)
            new_root.add_child(g2)
            self.root = new_root
        else:
            parent.children.remove(node)
            parent.add_child(g1)
            parent.add_child(g2)
            self._handle_overflow(parent)

    def _pick_seeds_quadratic(self, items: List[Union[Rectangle, RTreeNode]]) -> Tuple:
        """
        Heurística cuadrática: elige par con mayor desperdicio si se agrupan.
        """
        max_waste = -1
        seeds = (items[0], items[1])
        for i in range(len(items)):
            for j in range(i+1, len(items)):
                a = items[i].mbr if isinstance(items[i], RTreeNode) else items[i]
                b = items[j].mbr if isinstance(items[j], RTreeNode) else items[j]
                waste = Rectangle.union(a, b).area() - a.area() - b.area()
                if waste > max_waste:
                    max_waste = waste
                    seeds = (items[i], items[j])
        return seeds


In [5]:
from random import randrange, random, randint
num_points = 10
seen = set()
rects = []

for _ in range(num_points):
  min_x = randint(0, 20)
  min_y = randint(0, 20)
  max_x = randint(min_x + 20, 40)
  max_y = randint(min_y + 20, 40)

  key = (min_x, min_y, max_x, max_y)
  if key in seen:
    continue

  seen.add(key)
  rects.append(Rectangle(min_x, min_y, max_x, max_y))

In [6]:
for rect in rects :
  rect.show()

Las coordenadas son [(16, 18) , (38, 40)]
Su area es : 484
Las coordenadas son [(2, 2) , (29, 25)]
Su area es : 621
Las coordenadas son [(8, 9) , (35, 39)]
Su area es : 810
Las coordenadas son [(5, 4) , (36, 31)]
Su area es : 837
Las coordenadas son [(20, 1) , (40, 25)]
Su area es : 480
Las coordenadas son [(1, 9) , (31, 30)]
Su area es : 630
Las coordenadas son [(16, 10) , (37, 30)]
Su area es : 420
Las coordenadas son [(5, 5) , (32, 39)]
Su area es : 918
Las coordenadas son [(8, 18) , (36, 40)]
Su area es : 616
Las coordenadas son [(11, 1) , (33, 21)]
Su area es : 440


In [7]:
tree = RTreeChild(max_children=3, min_fill=0.4)

for rect in rects :
  tree.insert(rect)

Las coordenadas son [(16, 18) , (38, 40)]
Su area es : 484
Las coordenadas son [(20, 1) , (40, 25)]
Su area es : 480
Las coordenadas son [(16, 18) , (38, 40)]
Su area es : 484
Llega aca
Las coordenadas son [(20, 1) , (40, 25)]
Su area es : 480
Las coordenadas son [(16, 18) , (38, 40)]
Su area es : 484
Las coordenadas son [(20, 1) , (40, 25)]
Su area es : 480
Llega aca
Las coordenadas son [(16, 18) , (38, 40)]
Su area es : 484
